<a href="https://colab.research.google.com/github/akashgawali13/Dialogue-summerizer-NLP/blob/main/Dialogue_Summarizer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install Requirements
!pip install transformers sentence-transformers nltk spacy fpdf streamlit -q
!python -m nltk.downloader punkt
!python -m nltk.downloader punkt_tab
!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━

In [ ]:
# 2. Import Modules
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import nltk
import spacy
import re
from fpdf import FPDF


In [ ]:
# 3. Initialize Models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
ner_model = spacy.load("en_core_web_sm")
sim_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 4. Preprocessing
def preprocess_transcript(text):
    lines = [line.strip() for line in text.strip().split('\n') if line.strip()]
    dialogues = []
    for line in lines:
        match = re.match(r"(\w+):\s+(.*)", line)
        if match:
            speaker, content = match.groups()
            dialogues.append((speaker, content))
    return dialogues

In [ ]:
# 5. Summarization (supports multilingual model switch)
def summarize_chunks(dialogues, max_bullets=5, lang='en'):
    chunk_text = " ".join([f"{s}: {t}" for s, t in dialogues])
    if lang != 'en':
        summarizer_local = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")
        summary = summarizer_local(chunk_text, max_length=256, min_length=50, do_sample=False)[0]['summary_text']
    else:
        summary = summarizer(chunk_text, max_length=200, min_length=30, do_sample=False)[0]['summary_text']
    bullets = nltk.sent_tokenize(summary)
    return bullets[:max_bullets]

In [ ]:
# 6. Redundancy Removal
def remove_redundant(bullets):
    embeddings = sim_model.encode(bullets, convert_to_tensor=True)
    filtered = []
    for i, b in enumerate(bullets):
        if all(util.pytorch_cos_sim(embeddings[i], sim_model.encode(f)).item() < 0.8 for f in filtered):
            filtered.append(b)
    return filtered


In [ ]:
# 7. Named Entity Recognition
def extract_entities(text):
    doc = ner_model(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [ ]:
# 8. Action Item Extraction (rule-based)
def extract_actions(dialogues):
    actions = []
    for speaker, sentence in dialogues:
        if re.search(r"\b(should|let's|need to|must|have to|please)\b", sentence, re.I):
            actions.append(f"{speaker}: {sentence}")
    return actions

In [ ]:
# 9. Export to PDF
def export_to_pdf(bullets, actions, filename='summary.pdf'):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Conversation Summary", ln=True, align='C')

    pdf.set_font("Arial", 'B', 12)
    pdf.cell(200, 10, txt="Summary Bullets:", ln=True)
    pdf.set_font("Arial", size=12)
    for bullet in bullets:
        pdf.multi_cell(0, 10, f"• {bullet}")

    pdf.set_font("Arial", 'B', 12)
    pdf.cell(200, 10, txt="Action Items:", ln=True)
    pdf.set_font("Arial", size=12)
    for action in actions:
        pdf.multi_cell(0, 10, f"- {action}")

    pdf.output(filename)

In [ ]:
# 10. Export to Markdown
def export_to_md(bullets, actions, filename='summary.md'):
    with open(filename, 'w') as f:
        f.write("# Conversation Summary\n\n## Bullet Points:\n")
        for b in bullets:
            f.write(f"- {b}\n")
        f.write("\n## Action Items:\n")
        for a in actions:
            f.write(f"- {a}\n")


In [ ]:
# 11. Real-time Simulation
def simulate_real_time(dialogue_stream, interval=2):
    import time
    history = []
    print("Real-time summary running... Press Ctrl+C to stop.")
    try:
        while dialogue_stream:
            new_line = dialogue_stream.pop(0)
            history.append(new_line)
            summary = summarize_chunks(history)
            summary = remove_redundant(summary)
            print("\n[REAL-TIME SUMMARY UPDATE]")
            for s in summary:
                print(f"• {s}")
            time.sleep(interval)
    except KeyboardInterrupt:
        print("\nStopped.")


In [ ]:

# 12. Demo Input
sample = """
Emma: Good morning everyone. Let’s start with updates on the onboarding module.
Liam: Sure. We’ve completed the walkthrough flow for new users, but the welcome email logic still needs testing.
Sophia: I tested it last night—emails are firing but the template has alignment issues.
Olivia: I’ll ask the design team to review the email template today.
Emma: Great. Please prioritize it so we can push it to staging by tomorrow.
Noah: Speaking of staging, the backup service failed during last night’s run. I’ll investigate and share a report by noon.
Liam: Was any data lost?
Noah: No, but we need to double-check the auto-recovery mechanism.
Sophia: I’ll coordinate with DevOps on that.
Emma: Please do. Now, about the analytics dashboard—what’s the status?
Olivia: The charts are loading fine, but the filtering isn’t working as expected.
Liam: I’ll debug that after lunch and update the issue ticket.
Emma: Thank you. We also have the partner integration with FinBank due next week.
Noah: Their API sandbox credentials were received yesterday. I’ll start testing the endpoints this afternoon.
Emma: Excellent. Please document anything odd in Postman and share it.
Sophia: Do we need to update the privacy policy due to this integration?
Emma: Yes, good point. Let’s review the policy draft on Thursday.
Liam: I’ll check with Legal and get their feedback by Wednesday.
Olivia: Quick note—the UI team needs a response on the login screen animation concept.
Emma: I saw it. Looks slick but feels heavy. Let’s simplify it.
Noah: I can reduce frame size and re-export it in Lottie format.
Emma: Perfect. Target for Friday?
Noah: Yes, doable.
Sophia: Also, we need to align on the Q3 hiring plan. The data team is understaffed.
Emma: Let’s schedule a 1-hour planning session on Monday.
Olivia: Can we invite HR to that call?
Emma: Absolutely. Include Priya from HR.
Liam: Oh, I almost forgot—Slack notifications for deployments aren’t triggering.
Noah: I’ll check the webhook setup today.
Emma: Thank you. Anything else?
Sophia: Should we reschedule the product demo with TechNova? Their PM was on leave.
Emma: Yes, move it to next Wednesday. Send a calendar update.
Olivia: I’ll handle that.
Emma: Lastly, reminder: performance reviews are due next Friday. Submit yours by EOD Tuesday.
Noah: Got it.
Liam: Acknowledged.
Sophia: Noted.
Olivia: Thanks, Emma.


"""


In [ ]:

# 13. Run Pipeline
dialogues = preprocess_transcript(sample)
bullets = summarize_chunks(dialogues, max_bullets=5)
bullets = remove_redundant(bullets)
actions = extract_actions(dialogues)
entities = extract_entities(sample)


Your max_length is set to 200, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


In [ ]:
# 14. Display Output
print("\n--- SUMMARY BULLETS ---")
for b in bullets:
    print(f"• {b}")

print("\n--- ACTION ITEMS ---")
for a in actions:
    print(f"- {a}")

print("\n--- NAMED ENTITIES ---")
for ent, label in entities:
    print(f"[{label}] {ent}")


--- SUMMARY BULLETS ---
• CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery.
• Please submit your best shots of New York for next week.
• Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.

--- ACTION ITEMS ---

--- NAMED ENTITIES ---
[TIME] morning
[TIME] last night
[PERSON] Olivia
[DATE] today
[PERSON] Emma
[DATE] tomorrow
[PERSON] Noah
[TIME] last night
[TIME] noon
[PERSON] Liam
[PERSON] Noah
[ORG] DevOps
[PERSON] Emma
[PERSON] Olivia
[PERSON] Liam
[PERSON] Emma
[ORG] FinBank
[DATE] next week
[PERSON] Noah
[DATE] yesterday
[TIME] this afternoon
[PERSON] Emma
[LOC] Postman
[PERSON] Emma
[DATE] Thursday
[PERSON] Liam
[ORG] Legal
[DATE] Wednesday
[ORG] Olivia: Quick
[ORG] UI
[PERSON] Emma
[PERSON] Noah
[PERSON] Lottie
[PERSON] Emma
[ORG] Target
[DATE] Friday
[PERSON] Noah
[PRODUCT] Q3
[PERSON] Emma
[TIME] 1-hour
[DATE] Monday
[PERSON] Olivia
[PERSON] Emma
[PERSON] Liam
[PERSON] Noah
[DATE] today
[PERSON] Emma
[PRODUCT] TechNova
[PERSON] Emma
[